In [3]:
"""
initialization on library
"""
import networkx as nx
import sys
import pandas as pd
from itertools import combinations

import numpy as np
import os
import NCITerminology as NCI
#%matplotlib inline
sys.path.append('/cellar/users/btsui/Project/METAMAP/code/metamap')
import sharedVariable as sh
import seaborn as sns
"""
paramters setting for pdf
"""
import matplotlib as mpl
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['legend.numpoints'] = 1
mpl.rc('xtick', labelsize=15) 
mpl.rc('ytick', labelsize=15) 
mpl.rcParams['figure.subplot.bottom']=0.3
mpl.rcParams['figure.subplot.left']=0.3
import pylab as plot
#params = {'legend.fontsize': 20}

params = {'font.size': 15,'lines.linewidth':3}
plot.rcParams.update(params)

## data directories


In [4]:
metaDataMappingSDir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP//input/allAttrib.v5.csv.pyc'
#sraDumpDir='/scratch/btsui/Data/SRA/DUMP/SRA_Run_Members.tab'
parentFilteredNciDfDir='./MappingData/parent_filtered_NciDf.csv'
parentFilteredNciDf_pickleDir='./MappingData/parent_filtered_NciDf.pickle'

In [5]:
!ls -alth /cellar/users/btsui/Data/nrnb01_nobackup/METAMAP//input/allAttrib.v5.csv.NCI.prefilter.pyc

-rw-r--r-- 1 btsui users 1.0G Jan 30 10:40 /cellar/users/btsui/Data/nrnb01_nobackup/METAMAP//input/allAttrib.v5.csv.NCI.prefilter.pyc


## paramters

In [6]:
nonSpecificTerms=[]#['Embryo','Embryonic Cell','Fetus','Cell','Tissue','Stem Cell','Cell Line','Organ','Central Nervous System Part']

indexSpacingCharKey='jomifdasfd'
attributeSpacingCharKey='bmioasdgais'

## data loading

In [7]:
cuiS=pd.read_pickle(metaDataMappingSDir)

In [8]:
#sra_dumpDf=pd.DataFrame.from_csv(sraDumpDir,sep='\t')

In [9]:
"""
dataInDir='/cellar/users/btsui/Data/nrnb01_nobackup/Data/SRA/MATRIX/DATA/'
processedSpecies=os.listdir(dataInDir)
processedSrrs=[]
for specie in processedSpecies:
    with open(dataInDir+specie+'/'+'allSRAmatrix.realign.row.txt') as f:
        specieProcessedSrr=f.read().split('\n')
        processedSrrs+=specieProcessedSrr
processedSrs=sra_dumpDf.Sample.loc[processedSrrs] 
"""

"\ndataInDir='/cellar/users/btsui/Data/nrnb01_nobackup/Data/SRA/MATRIX/DATA/'\nprocessedSpecies=os.listdir(dataInDir)\nprocessedSrrs=[]\nfor specie in processedSpecies:\n    with open(dataInDir+specie+'/'+'allSRAmatrix.realign.row.txt') as f:\n        specieProcessedSrr=f.read().split('\n')\n        processedSrrs+=specieProcessedSrr\nprocessedSrs=sra_dumpDf.Sample.loc[processedSrrs] \n"

## initialization

In [10]:
onto=NCI.NCITerminology()
is_a_rels=onto.relToEng[onto.relToEng.str.lower().str.contains('has[\w_]+_parent')].index.tolist()+['is_a','R82','R44','R52']
"""
nodes()
"""
validRels=is_a_rels
onto.setCellOntoEdgeSubset(validRels)

In [11]:
print ('egdge subsets: ',validRels)

('egdge subsets: ', ['A10', 'A11', 'is_a', 'R82', 'R44', 'R52'])


In [12]:
print ('is directed acyclic graph: ',nx.is_directed_acyclic_graph(onto.cellOntoSubset))

('is directed acyclic graph: ', False)


In [13]:
nonSpecificCUIs=onto.cleanedIdS[onto.cleanedIdS.isin(nonSpecificTerms)].index.tolist()
generalTerms=nonSpecificCUIs
#gtexTissueMappingS=pd.Series(gtexTissueMapping)

# data cleaning

#### make the index level fully hierachical

In [14]:
 #extract srs vector, attrib vector and CUI vector
l1Index=cuiS.index.get_level_values(0) 
srsIndex=l1Index.str.extract("([A-Z]RS\d+)"+indexSpacingCharKey,expand=False) 
tmpAttribIndex=l1Index.str.replace("[A-Z]RS\d+"+indexSpacingCharKey,"") #extract 
characterKeyRemovedIndex=tmpAttribIndex.str.replace(attributeSpacingCharKey,' ')
CUIIndex=cuiS.index.get_level_values(1)

In [15]:
#create the series
newMultI=pd.MultiIndex.from_arrays([srsIndex,characterKeyRemovedIndex,CUIIndex])
triLevCuiS=pd.Series(index=newMultI,data=cuiS.values)
triLevCuiS.index.names=['srs','attrib','CUI']

#### get NCI translatble CUIs 

In [16]:
uniqueCUIs=triLevCuiS.index.get_level_values('CUI').unique()

In [17]:

#get only CUIs with english and UMLS translation
intersectCUIs=np.intersect1d(uniqueCUIs,onto.UMLSToID[onto.UMLSToID.isin(onto.cleanedIdS.index)].index)

In [18]:
print ('CUI to NCI id conversion rate ',float(len(intersectCUIs))/len(uniqueCUIs))

('CUI to NCI id conversion rate ', 0.9641148325358851)


In [19]:
subCuiS=triLevCuiS[triLevCuiS.index.get_level_values('CUI').isin(intersectCUIs)]

In [20]:
NCIDf=subCuiS.reset_index(name='score')
NCIDf['NCI']=onto.UMLSToID.loc[NCIDf.CUI].values
NCIDf.score=np.abs(NCIDf.score)

In [21]:
prefiltered_dir=metaDataMappingSDir.replace('.pyc','.NCI.prefilter.pyc')

In [22]:
nciEngS=onto.cleanedIdS.loc[NCIDf.NCI]

In [23]:
NCIDf.loc[:,'NciEng']=nciEngS.values

In [24]:
#NCIDf.to_pickle(prefiltered_dir)

In [25]:
#cate_NCIDf=NCIDf.apply(lambda S:S.astype('category'),axis=0)

In [27]:
NCIDf.to_pickle(prefiltered_dir)

In [24]:
!ls -lah /cellar/users/btsui/Data/nrnb01_nobackup/METAMAP//input/allAttrib.v5.csv.NCI.prefilter.pyc

-rw-r--r-- 1 btsui users 1012M Jan 26 20:58 /cellar/users/btsui/Data/nrnb01_nobackup/METAMAP//input/allAttrib.v5.csv.NCI.prefilter.pyc


### get childern

NameError: name 'cate_NCIDf' is not defined

In [20]:
NCIDf.loc[:,'NciEng']=nciEngS.values

In [21]:
sortedNCIDf=NCIDf.sort_values(by=['srs','attrib'])

### filter parent terms

In [22]:
#withinGraphNodeDf=sortedNCIDf[sortedNCIDf.NCI.isin(onto.cellOntoSubset)&(sortedNCIDf.srs=='SRS945312')]
withinGraphNodeDf=sortedNCIDf[sortedNCIDf.NCI.isin(onto.cellOntoSubset)]#.NciEng.value_counts()#.mean()

In [23]:
#withinGraphNodeDf

In [24]:
#filter the parent terms
#srsList, attribList, CUIList, ScoreList=[],[],[],[]
#run until 540000
maskList=[] 
i=0
orderingTmpList=[]
for (srs,attrib), tmpCuiDf in withinGraphNodeDf.groupby(by=['srs','attrib'],sort=False):
    termList= tmpCuiDf['NCI'].values

    filterTerm=[]
    for a,b in combinations(termList,2) :
        if (nx.has_path(onto.cellOntoSubset,a,b)):
            filterTerm.append(b)
        if (nx.has_path(onto.cellOntoSubset,b,a)):
            filterTerm.append(a)
    maskList+=(~tmpCuiDf.NCI.isin(filterTerm)).tolist() #
    maskList
    orderingTmpList.append((srs,attrib))
    i+=1
    if i%10000==0:
        print i


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000


In [25]:
mask=np.array(maskList) #mask out the parent terms

In [26]:
parentFilteredNCIDf=withinGraphNodeDf[mask]

In [27]:
parentFilteredNCIDf.index=np.arange(parentFilteredNCIDf.shape[0])

In [28]:
reorderedDf=parentFilteredNCIDf[['srs','NciEng','attrib','score','NCI','CUI']]
reorderedDf.to_csv(parentFilteredNciDfDir)

In [30]:
tmpDf=reorderedDf.set_index(['srs','attrib','NciEng','NCI','CUI'])
tmpDf.index.names=['Sample','attribute','NCIT_Eng','NCIT_ID','NLM_CUI']
tmpDf.columns=['metamap_score']
tmpDf.to_pickle(parentFilteredNciDf_pickleDir)

NameError: name 'tmpDf' is not defined

In [ ]:
###

## scratch

In [ ]:
asdasdasd

In [ ]:
#withinGraphNodeDf[withinGraphNodeDf.srs=='SRS1090684']

### find set of edges that make sense

is graph acyclic?

In [ ]:
nx.is_directed_acyclic_graph(onto.cellOnto)

In [ ]:
#check number of attruisimple cycles


In [ ]:
#nciInTest='C20141'
#for in onto.cellOnto

In [ ]:
#srsS[srsS.index.get_level_values(1)=='genotype']
#srsS

debug: group by yield different ordering for the srs
if the order of group

In [ ]:
NCIDf.iloc[:len(mask)][mask]

In [ ]:
#import pandas as pd

In [ ]:
#tmpDf=pd.read_csv('./MappingData/prefilter_NCIDf.csv')

In [ ]:
#tmpDf.drop_duplicates(['srs','attrib']).shape